In [0]:
from pyspark.sql import functions as F

src_path = "s3://paysim-bronze-eng/ps_20174392719_1491204439457_log.csv"
df = (spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv(src_path))
print(f"Rows read: {df.count()}")
display(df.limit(10))



In [0]:
%sql
CREATE CATALOG IF NOT EXISTS keystone
MANAGED LOCATION 's3://paysim-bronze-eng/';

CREATE SCHEMA IF NOT EXISTS keystone.silver;
CREATE SCHEMA IF NOT EXISTS keystone.gold;


In [0]:
catalog = "keystone"
schema = "bronze"
table = "paysim_txn_raw"
target_table = f"{catalog}.{schema}.{table}"

(df.write
    .format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")
    .saveAsTable(target_table))


In [0]:
spark.sql(f"SELECT COUNT(*) FROM {target_table}").show()


In [0]:
from pyspark.sql.functions import input_file_name, current_timestamp
df = df.withColumn("_source_file", input_file_name()
         ).withColumn("_ingest_ts", current_timestamp())


In [0]:
"Ingestion completed. Bronze table ready."